In [1]:
# importing several libraries
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
import string
import nltk
import nltk.corpus
import sklearn

In [4]:
pip install wordcloud

  Using cached wordcloud-1.9.2-cp39-cp39-win_amd64.whl (153 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\priya\\anaconda3\\Lib\\site-packages\\wordcloud\\query_integral_image.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [5]:
from matplotlib import rcParams
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk import NaiveBayesClassifier
from nltk.corpus import wordnet 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestClassifier 
from nltk.classify.scikitlearn import SklearnClassifier

In [6]:
# Specify the path to the CSV file
data_path = r'C:\Users\priya\Documents\Web Mining\Womens Clothing E-Commerce Reviews.csv'

# Read the CSV file into a DataFrame
df=pd.read_csv(data_path)

# Display the first few rows of the DataFrame
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [7]:
# display column names
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [8]:
# check the shape of the dataset, rows and columns
df.shape

(23486, 11)

In [9]:
# Count the number of unique values in each column
df.nunique()

Unnamed: 0                 23486
Clothing ID                 1206
Age                           77
Title                      13993
Review Text                22634
Rating                         5
Recommended IND                2
Positive Feedback Count       82
Division Name                  3
Department Name                6
Class Name                    20
dtype: int64

In [10]:
# Count the number of nulls in each column
df.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [11]:
# removes any row that contains at least one NaN value
df.dropna(inplace=True)

In [12]:
# reseat the index after droping some rows 
df.reset_index(drop=True, inplace=True)
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [13]:
# Check for the missing values after droping the null values 
df.isnull().sum()

Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64

In [14]:
# drop unnecessary culomns 
df.drop(["Unnamed: 0", "Title", 'Clothing ID'], axis=1, inplace=True)

In [15]:
df.head()

,Age,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,60,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,47,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,49,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,39,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [16]:
# to remove spaces in columns and replace them with underscore 
df.columns= df.columns.str.replace(" ", "_")

In [17]:
# Create reviews Tuples  to store the words along the categorys  
reviews = []
# go through Recommended IND column and get the category and the index 
for (index , category) in enumerate(df.Recommended_IND):
    reviews.append((df.Review_Text[index],category)) # Store the review for spacific index with catogory inside texts array
# Print first 4
reviews[0:4]

[('I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
  0),
 ("I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments!",
  1),
 ('This shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!',
  1),
 ('I love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress 

In [18]:
# create lemmatizer 
lemmatizer = WordNetLemmatizer()

In [19]:
# Create a list of stopwords 
# Get English stopwords
stops= stopwords.words("english")
# Get a list of punctuation marks
punctuations=list(string.punctuation)
# Combine stopwords and punctuation lists
stops=stops+punctuations
# Print the combined list of stopwords and punctuation
stops, string.punctuation

(['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're",
  "you've",
  "you'll",
  "you'd",
  'your',
  'yours',
  'yourself',
  'yourselves',
  'he',
  'him',
  'his',
  'himself',
  'she',
  "she's",
  'her',
  'hers',
  'herself',
  'it',
  "it's",
  'its',
  'itself',
  'they',
  'them',
  'their',
  'theirs',
  'themselves',
  'what',
  'which',
  'who',
  'whom',
  'this',
  'that',
  "that'll",
  'these',
  'those',
  'am',
  'is',
  'are',
  'was',
  'were',
  'be',
  'been',
  'being',
  'have',
  'has',
  'had',
  'having',
  'do',
  'does',
  'did',
  'doing',
  'a',
  'an',
  'the',
  'and',
  'but',
  'if',
  'or',
  'because',
  'as',
  'until',
  'while',
  'of',
  'at',
  'by',
  'for',
  'with',
  'about',
  'against',
  'between',
  'into',
  'through',
  'during',
  'before',
  'after',
  'above',
  'below',
  'to',
  'from',
  'up',
  'down',
  'in',
  'out',
  'on',
  'off',
  'over',
  'under',
  'again',
  'further',
  'th

In [20]:
# business stopwords
business_stopwords= ["i'm","would", "look", "ordered", "wear", "fit", "one", "fits","bought", "looks", "also", "got", "think", "even",
                     "tried", "get", "could", "made","way","still", "runs","true" ,"right", "see","online","wearing", "however", "design","purchased","feel","go",
                     "enough","model","though","price","looked","person","better","first","going","try", "body" "bottom","time","many","looking","around","thought",
                     "make","wanted","saw","makes","went","find","found","buy","nan","i've", "since","seems","ok", "girl", "woman"]
stops= stops+business_stopwords
stops

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
# stopwords after updated with business_stopwords
len(stops)

273

In [22]:
# function to get the simpler version of pos tag  to use it in lemmitazation 
def get_simple_pos(tag):
    if tag.startswith('N') or tag.startswith('J'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN #default case

In [23]:
# function to return Lemmitzed words and cleaned from stop words  
def clean_review(words):
  output_words= []
  words_tokens= nltk.word_tokenize(words)  
  for word in words_tokens :

    if word.lower() not in stops:
      pos = pos_tag([word]) # get the part of speech of each word 
    
      clean_word=(lemmatizer.lemmatize(word.lower(), pos=get_simple_pos(tag)) for word, tag in pos)
      output_words.append(', '.join(map(str,clean_word )))
  return output_words

In [24]:
clean_review("My cats are running away from my arms")

['cat', 'run', 'away', 'arm']

In [25]:
cleaned_reviews= [(clean_review(text),category )for text,category  in reviews]

In [26]:
#check first 5 reviews 
cleaned_reviews[0:5]

[(['high',
   'hope',
   'dress',
   'really',
   'work',
   'initially',
   'petite',
   'small',
   'usual',
   'size',
   'outrageously',
   'small',
   'small',
   'fact',
   'zip',
   'reorder',
   'petite',
   'medium',
   'ok.',
   'overall',
   'top',
   'half',
   'comfortable',
   'nicely',
   'bottom',
   'half',
   'tight',
   'layer',
   'several',
   'somewhat',
   'cheap',
   'net',
   'layer',
   'imo',
   'major',
   'flaw',
   'net',
   'layer',
   'sewn',
   'directly',
   'zipper',
   'c'],
  0),
 (['love',
   'love',
   'love',
   'jumpsuit',
   "'s",
   'fun',
   'flirty',
   'fabulous',
   'every',
   'nothing',
   'great',
   'compliment'],
  1),
 (['shirt',
   'flatter',
   'due',
   'adjustable',
   'front',
   'tie',
   'perfect',
   'length',
   'legging',
   'sleeveless',
   'pair',
   'well',
   'cardigan',
   'love',
   'shirt'],
  1),
 (['love',
   'tracy',
   'reese',
   'dress',
   'petite',
   '5',
   'foot',
   'tall',
   'usually',
   '0p',
   'bran

In [ ]:
#.75% traning = 14746 and 25% testing = 19662-14746 =4916 
traning_words=cleaned_reviews[0:14746]
testing_words=cleaned_reviews[14746:]

In [27]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [31]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (e.g., 80% training, 20% testing)
training_words, testing_words = train_test_split(cleaned_reviews, test_size=0.2, random_state=42)


In [32]:
print(f"Training Data = {len(training_words)}")
print(f"Testing Data = {len(testing_words)}")

Training Data = 15729
Testing Data = 3933


In [34]:
# array contaning all words 
words_list=[]
for word in training_words:
        words_list+=word[0] # 0 index to get only the words 

In [35]:
# Total words in traning data 
len(words_list)

421713

In [36]:
#frequency distribution for all words 
freq= nltk.FreqDist(words_list)
# The .most_common() method lists the words which occur most frequently in the data along with the frequency
common=freq.most_common()
# features are an array of only the top words in word list without The number of words 
features= [i[0]for i in common]

In [37]:
print(len(common))
print(len(features))

13414
13414


In [38]:
# Most common 5 words 
common[0:5]

[('dress', 8621), ('size', 7660), ('love', 7094), ("n't", 5967), ('top', 5906)]

In [39]:
# List of 5 features 
features[0:5]

['dress', 'size', 'love', "n't", 'top']

In [52]:
# Visualizing the highest repeating words (features)

# wordcload is techniqe use to show which words are the most frequent 
wordCloud = WordCloud(background_color="white", max_words =3000).generate(str(features))

rcParams["figure.figsize"]= 10,20
plt.imshow(wordCloud)
plt.axis("off")
plt.show 

ValueError: Only supported for TrueType fonts

In [41]:
# function to return a set of the features with true or false 
def get_dict_for_feature(words):
  current_features={}
  words_set= set(words)
  for word in features:
    current_features[word] = word in words_set  # if word comes in words set it will return True otherwise False 
  return current_features

In [43]:
featuers_dic= get_dict_for_feature(training_words[0][0])

In [44]:
# Dictionary containing all words with True classification if the word is exist in each review otherwise false  
featuers_dic  

{'dress': False,
 'size': False,
 'love': False,
 "n't": False,
 'top': False,
 "'s": False,
 'like': False,
 'color': False,
 "'m": False,
 'great': True,
 '5': False,
 "''": False,
 'fabric': False,
 'small': False,
 'really': False,
 'perfect': False,
 'little': False,
 'flatter': False,
 'soft': False,
 'well': False,
 'back': False,
 'nice': False,
 'comfortable': True,
 'cute': False,
 'work': False,
 'bit': False,
 'material': False,
 'large': False,
 'beautiful': False,
 'shirt': False,
 'length': False,
 'sweater': True,
 'much': False,
 'jean': False,
 'petite': False,
 'long': False,
 'short': False,
 'waist': False,
 'skirt': False,
 'medium': False,
 'quality': False,
 'retailer': False,
 'pretty': False,
 'x': False,
 '...': False,
 'store': False,
 'usually': False,
 'sleeve': False,
 'pant': False,
 'good': False,
 'style': False,
 'cut': False,
 'big': False,
 'return': False,
 'black': False,
 'super': False,
 'picture': False,
 'say': False,
 '4': False,
 'need': Fal

In [45]:
# create dic for each review wich has feature with value and the category
training_words= [( get_dict_for_feature(words),category ) for words , category in training_words]
testing_words = [( get_dict_for_feature(words),category ) for words , category in testing_words]

In [46]:
training_words[1]

({'dress': False,
  'size': True,
  'love': False,
  "n't": False,
  'top': True,
  "'s": False,
  'like': False,
  'color': True,
  "'m": False,
  'great': True,
  '5': False,
  "''": False,
  'fabric': False,
  'small': False,
  'really': False,
  'perfect': False,
  'little': True,
  'flatter': False,
  'soft': False,
  'well': False,
  'back': False,
  'nice': False,
  'comfortable': False,
  'cute': False,
  'work': False,
  'bit': False,
  'material': False,
  'large': False,
  'beautiful': True,
  'shirt': False,
  'length': False,
  'sweater': False,
  'much': False,
  'jean': False,
  'petite': False,
  'long': True,
  'short': False,
  'waist': False,
  'skirt': False,
  'medium': False,
  'quality': False,
  'retailer': False,
  'pretty': False,
  'x': False,
  '...': True,
  'store': False,
  'usually': False,
  'sleeve': True,
  'pant': False,
  'good': False,
  'style': False,
  'cut': False,
  'big': False,
  'return': False,
  'black': False,
  'super': False,
  'pictur

In [47]:
# to the classifier we need to use NaiveBayesClassifier and pass the training words to it 
NB_classifier= NaiveBayesClassifier.train(training_words)
print("classifier accuracy percent:",(nltk.classify.accuracy(NB_classifier, training_words))*100)

classifier accuracy percent: 91.93209994278085


In [48]:
review_1 = "Super fast and responsive with any issues. Different style print option was great! Easy to order and a pleasure to have done business with. Looking forward to ordering more items! Thank you"
review_2= "I am thrilled with the quality & fit of the t-shirts& they were very nicely packaged too. I will definitely be re-ordering from you in the future. - Kristina - Spain"
review_3="Missing refunds. Returned parcel and got a date that I would get the refund by, five days after this date no refund. Contacted customer support and they advised I have to wait another 14 days. The service was very unhelpful and rude at times."
reviews = [review_1,review_2,review_3]


def test_custom_review(reviews_list, classifier):
    
    for idx,review in enumerate(reviews_list) : 
        custom_tokens = clean_review(review)
        print(f"The clean review is : "  , str(custom_tokens).replace('[','').replace(']',''))
        classifiers=classifier.classify(dict([token, True] for token in custom_tokens))
        if (classifiers == 1):
            pred = "Positive"
        else:
            pred = "Negative"
        print(f"Review number {idx +1 }  seems to be {pred} \n")

In [49]:
test_custom_review(reviews,NB_classifier)

The clean review is :  'super', 'fast', 'responsive', 'issue', 'different', 'style', 'print', 'option', 'great', 'easy', 'order', 'pleasure', 'do', 'business', 'forward', 'order', 'item', 'thank'
Review number 1  seems to be Positive 

The clean review is :  'thrill', 'quality', 't-shirt', 'nicely', 'package', 'definitely', 're-ordering', 'future', 'kristina', 'spain'
Review number 2  seems to be Positive 

The clean review is :  'miss', 'refund', 'return', 'parcel', 'date', 'refund', 'five', 'day', 'date', 'refund', 'contact', 'customer', 'support', 'advise', 'wait', 'another', '14', 'day', 'service', 'unhelpful', 'rude', 'time'
Review number 3  seems to be Negative 

